In [ ]:
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool
from tools import market_data_tool, mpc_execution_tool, specific_info_tool
import os

# Set your API Key
os.environ["OPENAI_API_KEY"] = "your-key-here"

model = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# Wrap our logic into a tool for the sub-agents
@tool
def trend_analysis_tool(ticker: str):
    """Calculates the historical drift (mu) for a ticker."""
    data = market_data_tool.invoke(ticker)
    # Simple logic: returns a mu value based on price change
    prices = data['prices']
    mu = (prices[-1] - prices[0]) / prices[0]
    return {"mu": mu, "prices": prices}

In [ ]:
# Trend Agent: Responsible for the mathematical 'mu'
trend_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Trend Analyst. Use market_data to calculate the drift (mu)."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

trendAgent = create_openai_functions_agent(model, [trend_analysis_tool], trend_prompt)
trendAgentExecuter = AgentExecutor(agent=trendAgent, tools=[trend_analysis_tool], verbose=True)

# Noise Agent: Responsible for Sentiment/External Factors
noise_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Sentiment Analyst. Search for news that might affect stock noise."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

noiseAgent = create_openai_functions_agent(model, [specific_info_tool], noise_prompt)
noiseAgentExecuter = AgentExecutor(agent=noiseAgent, tools=[specific_info_tool], verbose=True)

In [ ]:
@tool 
def trendAgentTool(ticker: str):
    """Calls the Trend Agent to get the mu (drift) value."""
    return trendAgentExecuter.invoke({"input": f"Analyze the trend for {ticker}"})

@tool 
def noiseAgentTool(ticker: str):
    """Calls the Noise Agent to get sentiment info."""
    return noiseAgentExecuter.invoke({"input": f"Analyze recent news for {ticker}"})

# The "Super" Prompt that connects everything to the MPC
sup_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are the Portfolio Supervisor. 
   Goal is to provide a trade recommendation for a ticker.
    1. Call market_data_tool to get prices.
    2. Call trendAgentTool to get the drift (mu).
    3. Call mpc_execution_tool using the 'mu' from the Trend Agent and 'prices' from market data.
    4. Summarize the final action (Buy/Sell/Hold) for the user."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

supAgent = create_openai_functions_agent(model, [trendAgentTool, noiseAgentTool, mpc_execution_tool, market_data_tool], sup_prompt)
supAgentExecuter = AgentExecutor(agent=supAgent, tools=[trendAgentTool, noiseAgentTool, mpc_execution_tool, market_data_tool], verbose=True)